In [1]:
import pandas as pd
from pathlib import Path
from ast import literal_eval

from dashboard.data_load import load_sorf_df_conformed, load_protein_feature_string_representations
from dashboard.etl import CACHE_DIR, DATA_DIR

2023-12-19 05:40:48.989 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-19 05:40:49.006 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-19 05:40:49.008 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-19 05:40:49.009 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-19 05:40:49.010 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-19 05:40:49.011 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-19 05:40:49.012 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-19 05:40:49.013 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-

In [2]:
sorf_df = load_sorf_df_conformed()

2023-12-19 05:40:49.140 
  command:

    streamlit run /home/ubuntu/miniconda3/envs/veliadash_updated/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2023-12-19 05:40:49.141 No runtime found, using MemoryCacheStorageManager


In [3]:
def add_temp_feature_strings_info(df):
    """
    """
    feature_df = pd.read_csv(CACHE_DIR.joinpath('protein_data', 'sequence_features_strings.csv'), index_col=0)

    signal_cols = ['SignalP 6slow', 'SignalP 4.1', 'SignalP 5b', 'Deepsig']
    nonsignal_seqs = []

    for i, row in feature_df.iterrows():
        nonsignal_aa = ''
        for col in signal_cols:
            if row[col].startswith('S'):
                signal_len = row[col].count('S')
                nonsignal_aa = row['Sequence'][signal_len:-1]
                break
        nonsignal_seqs.append(nonsignal_aa)

    feature_df['nonsignal_seqs'] = nonsignal_seqs

    feature_df['DeepTMHMM_prediction'] = feature_df.apply(lambda x: 'M' in x['DeepTMHMM'] or 'B' in x['DeepTMHMM'], axis=1)
    feature_df['DeepTMHMM_length'] = feature_df.apply(lambda x: x['DeepTMHMM'].count('M'), axis=1)

    feature_cols = ['nonsignal_seqs', 'DeepTMHMM_prediction', 'DeepTMHMM_length']
    df.merge(feature_df[feature_cols], left_index=True, right_index=True, how='left')

    return df
    

In [4]:
df = add_temp_feature_strings_info(sorf_df)

In [5]:
df

,show_details,vtx_id,aa_length,screening_phase_id,screening_phase,ucsc_track,source,orf_xrefs,protein_xrefs,gene_xrefs,...,tblastn_hit_id,tblastn_description,tblastn_score,tblastn_query_coverage,tblastn_align_length,tblastn_align_identity,tblastn_gaps,tblastn_evalue,Ribo-Seq RPKM Support,Ribo-Seq sORF
vtx_id,,,,,,,,,,,,,,,,,,,,,
VTX-0852017,False,VTX-0852017,54,-1,Phase 1,chr1:-1--1,[velia_phase1_Bona fide],"[NaN, PIGBOS1 NT control ]",[PIGBOS1 NT control ],,...,NM_001308425.1,NM_001308425.1 Mus musculus Pigb opposite stra...,193.0,91.0,49.0,71.429,0.0,1.380000e-19,False,True
VTX-0852018,False,VTX-0852018,53,-1,Phase 1,chr1:-1--1,[velia_phase1_Bona fide],"[HOXB-AS3 NT control , NaN]","[HOXB-AS3 NT control , HOXB-AS3 NT control , N...",,...,NM_001355211.2,NM_001355211.2 Mus musculus zinc finger and BT...,60.0,77.0,41.0,34.146,0.0,4.000000e+00,False,True
VTX-0852019,False,VTX-0852019,45,Phase 2_154,Phase 2,chr1:-1--1,[velia_phase2_Bonafide_Bianca],"[Phase 2_154, U9096HI220-282, ZBTB37]","[Phase 2_154, Phase 4_352, sORF2154, sORF4352,...",,...,XM_011238812.3,XM_011238812.3 PREDICTED: Mus musculus zinc fi...,123.0,96.0,28.0,92.857,0.0,1.250000e-10,False,True
VTX-0852020,False,VTX-0852020,42,Phase 2_155,Phase 2,chr1:-1--1,[velia_phase2_Bianca_Chen],"[17-242, 6xhis-HSA-HRV3C-sORF2155_1_02, Phase ...","[17-242, 57729, 6xhis-HSA-HRV3C-sORF2155_1_02,...",,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True
VTX-0852021,False,VTX-0852021,59,Phase 2_156,Phase 2,chr1:-1--1,[velia_phase2_Bianca_Chen],"[Phase 2_156, U312HHH170-182, ZFAND2A_1199767_...","[c7riboseqorf5, Phase 2_156, Phase 4_727, sORF...",,...,XR_003951873.1,XR_003951873.1 PREDICTED: Mus musculus predict...,62.0,51.0,30.0,40.000,0.0,2.500000e+00,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VTX-0851963,False,VTX-0851963,79,Phase 3_94,Phase 3,chr1:120005395-120029951,[velia_phase3_nan],"[NOTCH2/ENST00000579475.7/mRNA/5'UTR, NOTCH2/N...","[NOTCH2/ENST00000579475.7/mRNA/5'UTR, NOTCH2/N...",ENSG00000134250.21;HGNC:7882;NOTCH2;4853;7882;...,...,NM_010928.2,"NM_010928.2 Mus musculus notch 2 (Notch2), mRNA",224.0,95.0,75.0,66.667,0.0,5.680000e-22,True,False
VTX-0851964,False,VTX-0851964,81,U738YHG260-508,Phase 3,chr12:6388403-6388942,[velia_phase3_nan],"[LTBR/ENST00000543542.1/ncRNA/-, U738YHG260-508]","[LTBR/ENST00000543542.1/ncRNA/-, NaN, U738YHG2...",ENSG00000111321.11;HGNC:6718;LTBR;4055;6718;ge...,...,NM_010736.5,NM_010736.5 Mus musculus lymphotoxin B recepto...,96.0,28.0,23.0,69.565,0.0,1.410000e-04,False,False
VTX-0851965,False,VTX-0851965,81,U738YHG260-510,Phase 3,chr4:125487463-125490005,[velia_phase3_nan],"[FAT4/NM_001291285.1/mRNA/CDS, U738YHG260-510]","[FAT4/NM_001291285.1/mRNA/CDS, NaN, U738YHG260...",ENSG00000196159.14;FAT4;HGNC:23109;23109;79633...,...,NM_183221.4,NM_183221.4 Mus musculus FAT atypical cadherin...,174.0,98.0,79.0,54.430,0.0,3.200000e-15,False,False


In [7]:
sorf_df.columns

Index(['show_details', 'vtx_id', 'aa_length', 'screening_phase_id',
       'screening_phase', 'ucsc_track', 'source', 'orf_xrefs', 'protein_xrefs',
       'gene_xrefs', 'transcript_xrefs', 'transcripts_exact', 'aa', 'nucl',
       'index_copy', 'genscript_id', 'chr', 'strand', 'start', 'end',
       'chrom_starts', 'block_sizes', 'phases', 'blastp_score',
       'blastp_query_coverage', 'blastp_align_length', 'blastp_gaps',
       'blastp_align_identity', 'blastp_subject', 'blastp_hit_description',
       'Deepsig_score', 'SignalP 6slow_score', 'SignalP 5b_score',
       'SignalP 4.1_score', 'Deepsig_cut', 'SignalP 6slow_cut',
       'SignalP 5b_cut', 'SignalP 4.1_cut', 'Phobius', 'DeepTMHMM', 'trans1',
       'trans2', 'trans3', 'sec1', 'sec2', 'sec3', 'translated_mean',
       'secreted_mean', 'secreted', 'translated', 'phylocsf_58m_avg',
       'phylocsf_58m_max', 'phylocsf_58m_min', 'ESMFold plddt 90th percentile',
       'manual_riboseq', 'MS_evidence', 'MS or Riboseq', 'swissprot

In [ ]:
view_cols = [
    'show_details', 'vtx_id', 'aa_length', 'ucsc_track', 'source', 
    'screening_phase_id', 'screening_phase', 'genscript_id',
    'orf_xrefs', 'protein_xrefs', 'gene_xrefs', 'transcript_xrefs', 
    'transcripts_exact', 'aa', 'nucl', 'chr', 'strand', 'start', 'end',
    'chrom_starts', 'block_sizes', 'phases', 
    'blastp_query_coverage', 'blastp_align_length', 'blastp_align_identity', 
    'blastp_subject', 'blastp_hit_description',
    'tblastn_hit_id', 'tblastn_description',  'tblastn_query_coverage',
    'tblastn_align_length', 'tblastn_align_identity', 
    'Deepsig_score', 'SignalP 6slow_score', 'SignalP 5b_score', 'SignalP 4.1_score', 
    'Deepsig_cut', 'SignalP 6slow_cut', 'SignalP 5b_cut', 'SignalP 4.1_cut', 
    'Phobius', 'DeepTMHMM', 
    'translated_mean', 'secreted_mean', 'secreted', 'translated', 
    'phylocsf_58m_avg', 'phylocsf_58m_max', 'phylocsf_58m_min', 'ESMFold plddt 90th percentile',
    'MS_evidence', 'swissprot_isoform', 'ensembl_isoform', 'refseq_isoform', 
    'Ribo-Seq RPKM Support', 'Ribo-Seq sORF']

In [42]:
with open('../cache_updates/all_vtx_gencode_231218.txt', 'w') as outfile:
    for i, row in df.iterrows():
        outfile.write(f"{row.name}\n")



'VTX-0851967'